## Simulator for two link arm for 2023 FRC competition

This notebook was written to be followed by someone without extensive Python
experience and knowledge of High School trigonometry. The goal was to allow
someone new to the concepts to follow the flow and the math.

The dimensions are unitless here so they can be in any unit that works.

The angles are shown in degrees but converted to radians before calculation.

In [1]:
from ipycanvas import Canvas, hold_canvas

In [2]:
from ipywidgets import widgets, HBox, VBox, Layout, interact

In [3]:
import math

In [4]:
# You should be able to change this size for your preference
robot_canvas_width = 500
robot_canvas_height = 300
    
robot_canvas = Canvas(width=robot_canvas_width, height=robot_canvas_height)

In [5]:
def draw_robot_canvas(canvas, rf):
    """
    Given a canvas to draw on and the points of a robot figure,
    draw a stick representation of the robot and arm.
    """
    with hold_canvas():
        canvas.clear()
        canvas.reset_transform()

        # The origin of the canvas is the top which is upside down so we will
        # move it to the bottom left corner and invert the y-axis
        canvas.translate(0, canvas.height)
        canvas.scale(3.0, -3.0)

        canvas.line_width = 7
        canvas.line_cap = 'round'
        
        # Draw a shallow back triangle for robot base
        canvas.begin_path()
        canvas.stroke_style = "black"
        canvas.stroke_lines([rf.b1, rf.b0, rf.j0])
        canvas.stroke()
        
        # Draw first link (j0-j1)
        canvas.begin_path()
        canvas.stroke_style = "green"
        canvas.stroke_lines([rf.j0, rf.j1])
        canvas.stroke()

        # Draw second link (j1-j2)
        canvas.line_width = 5
        canvas.begin_path()
        canvas.stroke_style = "blue"
        canvas.stroke_lines([rf.j1, rf.j2])
        canvas.stroke()
        
        # Draw a line that is 48 inches from the robot base
        canvas.line_width = 1
        canvas.begin_path()
        canvas.stroke_style = "red"
        canvas.stroke_lines([rf.limit1, rf.limit2])
        canvas.stroke()
        


In [6]:
class robot_configuration:
    """
    This class holds the robot dimensions and the joint state angles.
    The user will need to update the dimensions to match the robot.
    """
    # This tracks the robots location on the floor
    base_x = 100

    # This is the height of the first arm jpoint above the floor
    base_height = 5
    
    # This is the length to the back of the robot from the front
    base_length = 32
    
    # This is the rule limit for the maximum distance from the robot
    limit = 48
    
    # Length of the first robot link
    l1 = 32
    j0_degrees = 180
    
    # Length of the second robot link
    l2 = 32
    j1_degrees = 180

# Create an instance of the class
rc = robot_configuration()

In [7]:
class robot_figure:
    """
    This class provides the task space representation of the key robot
    points for doing range checking and drawing. It depends on the
    robot configuration.
    """
    
    def __init__(self, robot_configuration):
        self.rc = robot_configuration
        self.b0 = [0,0] # Point at the front of the robot
        self.b1 = [0,0] # Point at the back of the robot
        self.j0 = [0,0] # Attachment of the arm
        self.j1 = [0,0] # End of first link
        self.j2 = [0,0] # End of second link
        self.limit1 = [0,0] # bottom of line for extension limit
        self.limit2 = [0,0] # top of line for extension limit
        
        self.d2r = math.pi/180 # convert degrees to radians
    
    def update_locations(self):
        #Draw the base first and the j0 (arm) attachment point
        self.b0 = [self.rc.base_x, 20]
        self.b1 = [self.rc.base_x - self.rc.base_length, 20]

        # Draw a line 48 inches in front of the robot
        self.limit1 = [self.b0[0] + self.rc.limit, self.b0[1]]
        self.limit2 = [self.b0[0] + self.rc.limit, self.b0[1]+200]

        self.j0 = [self.b0[0], self.b0[1]+self.rc.base_height]

        # Add the first link to the base
        self.j1[0] = self.j0[0] + self.rc.l1 * math.cos(self.rc.j0_degrees*self.d2r)
        self.j1[1] = self.j0[1] + self.rc.l1 * math.sin(self.rc.j0_degrees*self.d2r)
        
        # Add the second link to the first link
        # The second joint angle is relative to the first joint angle so they are added
        self.j2[0] = self.j1[0] + self.rc.l2 * math.cos((self.rc.j0_degrees+self.rc.j1_degrees)*self.d2r)
        self.j2[1] = self.j1[1] + self.rc.l2 * math.sin((self.rc.j0_degrees+self.rc.j1_degrees)*self.d2r)


# Create an instance of the class
rf = robot_figure(rc)


# Brief explanation of user interface code below

The "@interact" decorator creates a display widget that sends a value to 
the robot configuration class. After the robot configuration is updated,
the robot figure class recalculates the forward kinematics of the robot.

The draw_canvas method is then called to use the forward kinematics to
draw the robot.

In [8]:
# Provide a widget that goes from -180 to 180 degrees
@interact(J0_angle=(-180.0,180.0))

# Handle a change to the J0 joint
def j0_change(J0_angle=180):
    rc.j0_degrees = J0_angle
    rf.update_locations()
    draw_robot_canvas(robot_canvas, rf)
    
@interact(J1_angle=(-180.0,180.0))

# Handle a change to the J1 joint
def j1_change(J1_angle=180):
    rc.j1_degrees = J1_angle
    rf.update_locations()
    draw_robot_canvas(robot_canvas, rf)

interactive(children=(FloatSlider(value=180.0, description='J0_angle', max=180.0, min=-180.0), Output()), _dom…

interactive(children=(FloatSlider(value=180.0, description='J1_angle', max=180.0, min=-180.0), Output()), _dom…

In [9]:
robot_canvas

Canvas(height=300, width=500)